Реализовать адаптивный бустинг использующий Логистическую Регрессию и меру ошибок LogLoss. Сравнить с точностью адаптивного бустинга на деревьях решений. Для сбора предсказаний можно использовать ту же функцию predict что и для бустинга на деревьях
Примечания: в LogLoss необходимо передавать не предсказания полученные с помощью clf.predict(...), а вероятность, полученную с помощью clf.predict_proba(...)[:, 1]

In [9]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt

from sklearn import model_selection
import numpy as np

In [10]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [11]:
#функция подсчета ошибки
def get_error(pred, y):
    return sum(pred != y) / len(y)

In [12]:
#реализация меры ошибок LogLoss
def log_loss(pred, y): 
    return -np.sum(y*np.log2(pred)+(1-y)*np.log2(1-pred))/len(y)

In [14]:
def adaboost(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Начальные веса
    w = np.ones(n_objects) / n_objects

    # список моделей с весами
    models = []

    for n in range(N):
        # Зададим логистическую регрессию и обучим ее
        clf = LR()
        import warnings
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            clf.fit(X, y, w)

        predictions = clf.predict(X)
        predictions_proba = clf.predict_proba(X)[:, 1]
        e = log_loss(predictions_proba, y)
        
        if e >= 1:  
            break

        # Вычислим вес 
        alpha = 0.5 * np.log((1 - e) / e)

        # Найдем индексы правильно классифицированных элементов
        match = predictions == y

        # Увеличим веса для неправильно классифицированных элементов
        w[~match] *= np.exp(alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим модель с весом в список
        models.append((alpha, clf))
    
    return models

In [15]:
#обучим алгоритм из 50 моделей
N = 50

models = adaboost(X_train, y_train, N)

In [16]:
#осуществим предсказание
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
    
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

    
  

print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 93.192


In [17]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 95.105


In [ ]:
#Вывод: на тестовой выборке точность адаптивного бустинга на деревьях решений
#ниже, чем точность адаптивного бустинга на логистической регрессии.

Реализовать специальную функцию predict для бустинга на логистической регрессии выводящую предсказания по формуле:  𝑃𝑟𝑒𝑑𝑖𝑐𝑡𝑖𝑜𝑛𝑠=𝑠𝑖𝑔𝑛(𝑆𝑐𝑜𝑟𝑒𝑏𝑢𝑠𝑡) , где sign равен единице для положительных и нулю для отрицательных значений, а  𝑆𝑐𝑜𝑟𝑒𝑏𝑢𝑠𝑡=∑𝛼𝑖𝑆𝑐𝑜𝑟𝑒𝑖 . Баллы выдаваемые каждой моделью  𝑆𝑐𝑜𝑟𝑒𝑖  можно найти при помощи вызова метода decision_function на моделе

In [18]:
def predict_lr(X, models):
   
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects))
    
    for alpha, clf in models:
        prediction = clf.decision_function(X)
       
        y_pred += alpha * prediction
    
    
    
    return (y_pred>0).astype(int)

    
  

print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict_lr(X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 93.897


In [19]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict_lr(X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 94.406
